In [106]:
from math import sqrt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, r2_score

In [95]:
def mse(y_pred, y_test):
    return np.mean((y_pred - y_test) ** 2)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [96]:
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights_input = self.__init_weights((input_size, hidden_size))  # матрица весов входного слоя
        self.weights_hidden = self.__init_weights((hidden_size, hidden_size)) # матрица весов рекуррентного слоя
        self.weights_output = self.__init_weights((hidden_size, output_size)) # матрица весов выходного слоя
        self.hidden_biases = np.zeros((1, hidden_size))
        self.output_biases = np.zeros((1, output_size))
        self.hidden_state = np.zeros((1, hidden_size))                        # предыдущее состояние рекуррентного слоя

    def __forward_propagation(self, input_vector):
        self.hidden_state = sigmoid(np.dot(input_vector, self.weights_input) + np.dot(self.hidden_state, self.weights_hidden) + self.hidden_biases)
        output = np.dot(self.hidden_state, self.weights_output) + self.output_biases
        return output
    
    def predict(self, x):
        return self.__forward_propagation(x)

    def __backward_propagation(self, input_vector, output, target, learning_rate):
        output_error = 2 * (output - target) / target.shape[0]  # вычисление ошибки на выходном слое
        output_grad = np.dot(self.hidden_state.T, output_error)  # вычисление градиентов для весов выходного слоя (V)
        by_grad = np.sum(output_error, axis=0, keepdims=True)  # вычисление градиентов для смещения выходного слоя (V)
        hidden_error = np.dot(output_error, self.weights_input.T) * sigmoid_derivative(self.hidden_state) # вычисление ошибки на скрытом слое

        input_grad = np.dot(input_vector.T, hidden_error)  # вычисление градиентов для весов входного слоя

        hidden_grad = np.dot(self.hidden_state.T, hidden_error)  # вычисление градиентов для рекуррентных весов

        bh_grad = np.sum(hidden_error, axis=0, keepdims=True)         # вычисление градиентов для смещения скрытого слоя

        # обновление весов и смещений
        self.weights_output -= learning_rate * output_grad
        self.output_biases -= learning_rate * by_grad
        self.weights_input -= learning_rate * input_grad
        self.weights_hidden -= learning_rate * hidden_grad
        self.hidden_biases -= learning_rate * bh_grad

    def train(self, x_train, y_train, epochs=100, learning_rate=0.01, batch_size=32):
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                batch_x, batch_y = x_train[i:i+batch_size], y_train[i:i+batch_size]
                outputs, states = [], [] # Список выходов и скрытых состояний для всех примеров в батче.

                # Вычисляет выходной вектор для каждого входного вектора x в батче
                for x in batch_x:
                    out = self.__forward_propagation(x.reshape(1, -1))
                    outputs.append(out)

                outputs = np.vstack(outputs)
                batch_y = batch_y.reshape(-1, 1)

                # Обратное распространение для каждого примера в батче
                for j in range(batch_x.shape[0]):
                    self.__backward_propagation(batch_x[j].reshape(1, -1), outputs[j].reshape(1, -1), batch_y[j].reshape(1, -1), learning_rate)


    def __init_weights(self, size):
        return np.random.uniform(-1, 1, size)

In [97]:
df = pd.read_csv('resources/Clear_steel_industry_data.csv')

In [98]:
x = df.drop('Usage_kWh', axis=1).values
y = df['Usage_kWh'].values.reshape(-1, 1)

In [99]:
# Нормализация признаков и целевой переменной
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x_scaled = scaler_x.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)

In [100]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, shuffle=False, random_state=42)

In [101]:
rnn = RNN(x_train.shape[1], 10, 1)
rnn.train(x_train, y_train, epochs=500, learning_rate=0.001, batch_size=16)

In [104]:
def test_model(rnn, x_test, y_test):
    predictions = [rnn.predict(x.reshape(1, -1)) for x in x_test]
    predictions = np.array(predictions).reshape(-1, 1)

    # Обратное масштабирование предсказанных значений
    y_test_inverse = scaler_y.inverse_transform(y_test)
    predictions_inverse = scaler_y.inverse_transform(predictions)
    print("R2 Score:\t", r2_score(y_test_inverse, predictions_inverse))
    print("RMSE:\t\t", root_mean_squared_error(y_test_inverse, predictions_inverse))

In [107]:
test_model(rnn, x_test, y_test)

R2 Score:	 0.9461760134782984
RMSE:		 7.278287033949173
